In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

In [2]:
# Create a new dataset and import the data in Pascal VOC format in one step
dataset = fo.Dataset.from_dir(
    dataset_dir='Images',
    dataset_type=fo.types.VOCDetectionDataset,
    name="cctv_gun_hackathon"
)


 100% |███████████████| 5149/5149 [15.3s elapsed, 0s remaining, 414.2 samples/s]      


In [ ]:
model = foz.load_zoo_model("faster-rcnn-inception-resnet-atrous-v2-lowproposals-coco-tf")
resnet = foz.load_zoo_model("centernet-resnet101-v1-fpn-512-coco-tf2")

In [ ]:
dataset.apply_model(model, label_field="predictions")

In [3]:
dataset = fo.load_dataset("cctv_gun_hackathon")

In [ ]:
# Launch the FiftyOne App to visualize your dataset
session = fo.launch_app(dataset)


In [12]:
dataset.get_field_schema()

OrderedDict([('id', <fiftyone.core.fields.ObjectIdField at 0x2d224f794e0>),
             ('filepath', <fiftyone.core.fields.StringField at 0x2d224f79000>),
             ('tags', <fiftyone.core.fields.ListField at 0x2d224f7e980>),
             ('metadata',
              <fiftyone.core.fields.EmbeddedDocumentField at 0x2d224f79600>),
             ('created_at',
              <fiftyone.core.fields.DateTimeField at 0x2d224f7fca0>),
             ('last_modified_at',
              <fiftyone.core.fields.DateTimeField at 0x2d224f78310>),
             ('ground_truth',
              <fiftyone.core.fields.EmbeddedDocumentField at 0x2d224f7ee00>)])

In [10]:
yolo_model = foz.load_zoo_model("yolo11n-coco-torch")

In [ ]:
import fiftyone.brain as fob

In [4]:
view = dataset.select_fields("ground_truth").filter_labels("ground_truth", {"label": "Handgun"})
print(view)

Dataset:     cctv_gun_hackathon
Media type:  image
Num samples: 1534
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. SelectFields(field_names=['ground_truth'], meta_filter=None)
    2. FilterLabels(field='ground_truth', filter={'label': 'Handgun'}, only_matches=True, trajectories=False)


In [ ]:
print(dataset.select_fields("ground_truth").take(1))

Dataset:     cctv_gun_hackathon
Media type:  image
Num samples: 1
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. SelectFields(field_names=['ground_truth'], meta_filter=None)
    2. Take(size=1, seed=None)


In [5]:
gt_viz = fob.compute_visualization(dataset, patches_field="ground_truth", brain_key="gt_viz1", method="tsne") 

Computing patch embeddings...
 100% |███████████████| 5149/5149 [5.3m elapsed, 0s remaining, 13.0 samples/s]      
Generating visualization...


c:\Users\maria\Documents\GitHub\hackathon-gun-detection\.venv\lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2721 samples in 0.007s...
[t-SNE] Computed neighbors for 2721 samples in 1.292s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2721
[t-SNE] Computed conditional probabilities for sample 2000 / 2721
[t-SNE] Computed conditional probabilities for sample 2721 / 2721
[t-SNE] Mean sigma: 1.291227
[t-SNE] Computed conditional probabilities in 0.148s
[t-SNE] Iteration 50: error = 79.8543396, gradient norm = 0.0019775 (50 iterations in 3.017s)
[t-SNE] Iteration 100: error = 79.5112534, gradient norm = 0.0111010 (50 iterations in 1.849s)
[t-SNE] Iteration 150: error = 78.7934341, gradient norm = 0.0024168 (50 iterations in 1.567s)
[t-SNE] Iteration 200: error = 78.7911987, gradient norm = 0.0014051 (50 iterations in 1.242s)
[t-SNE] Iteration 250: error = 78.7876892, gradient norm = 0.0033400 (50 iterations in 1.054s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 78.787689
[t-SNE] Iteration 300: erro

In [13]:
session = fo.launch_app(dataset)

In [11]:
dataset.apply_model(
    yolo_model,
    label_field="predictions_yolo",
    confidence_thresh=0.5,  # optional confidence threshold for filtering predictions
)

 100% |███████████████| 5149/5149 [23.9m elapsed, 0s remaining, 4.6 samples/s]      


In [ ]:
embeddings = dataset.compute_embeddings(model=yolo_model)